# Marketing campaigns Subscription Predictor
Shiying Wang,  Karlos Muradyan, Gaurav Sinha

# Introduction

Businesses provide numerous services to their clients. It becomes important for them to know whether their clients need those services. Banking is one sector which provides numerous services to their clients. If a client subscribes to their service, it increases revenue for the bank. One such service offered by banks is *Term Deposit*. A term deposit is a form of deposit in which money is held for a fixed duration of time with the financial institution. A client will subscribe to a term deposit or not is dependent on a large number of features of a client. Banks generally have this information of clients which can help to predict whether a client will subscribe to a term deposit or not. This is an interesting problem which can be solved by analysing the data and building model to predict such clients behaviour. 

In this project, we will analyze a *Bank Marketing* data of a Portuguese banking institution, and predict whether the a client will subscribe for a term deposit from the marketing campaign. 


# Methods

## Data

The dataset we chose is Bank Marketing data of a Portugese banking instituion collected from [UCI Machine learning Repository](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing). The dataset is publicly available for research, the details of which are described in [Moro et al., 2014](#Reference). Detail of the source is provided in the reference session.

The dataset has 4521 observations, 16 features and a response variable `y` which indicates if the client subscribed a term deposit or not (binary: 'yes','no'). The classes are imbalance: there are 4000 no and 521 yes. There are 0 observations with missing values in the data set. 

## Exploratory Data Analysis

We split the data as 80% training dataset and 20% test data. We used the training dataset to perform the analysis. Figure 1 is the proportion of the response variable y. 

<img src="../reports/proportion_of_class.png">

Figure 1. Proportion of response variable: whether the client subscribed.

From the plot, we can see that the classes are highly imbalance: around 90% of people did not subscribe after the marketing campaign, and only around 10% subscribed after the marketing campaign. The highly imbalance dataset would cause the problem that the model only predicted the majority classes. We will discuss how to solve it in the [Model](#Model) section.

This dataset has combination of numerical and categorical features. We explored them respectively. Figure 2 Figure 3 shows the relationship between the numerical features. 

<img src="../reports/kendall_corr_matrix.png" width="400" height="400"> <img src="../reports/pearson_corr_matrix.png" width="400" height="400">

Figure 2: kendall and pearson correlation matrix

From the kendall and pearson correlation matrix, most features does not have correlation with others. The only big correlation between numerical features is between pdays and previous. Kendall's correlation coefficient gives value greater than 0.9

<img src="../reports/pairplot_numeric.png">
Figure 3: pairplot of numerical features.

From Figure 3, we found some interested patterns: for people who did not subscribed (blue), the density plot for `duration` is more concentrated on the left. This indicates that this class tend to has lower duration of last contact. Most of the people who subscribed after the campaign (red) haven't had previous contacts before. Those indicates that `duration`, `pdays` and `previous` might be important features.

For categorical features, we looked at number of counts for each class to see some common characteristics of clients in our dataset.

<img src="../reports/count_of_cat_features.png">

Figure 4: counts of categorical features.

From the figure 4, we observed some common characteristics:
- In this dataset:
    - Most clients have job type management or blue-collar.
    - Most clients are married.
    - Most clients have secondary education level.
    - Almost all clients does not have credit in default.
    - A big portion of clients have housing loan.
    - Most clients does not have personal loan.
    - Clients are mostly contacted by cellular.
    - Most clients do not have previous campaign before. 

- For those clients who subscribe the service after campaign:
    - Most of them are management or technician.
    - Most of them are married, and many of them are single.
    - Most of them have secondary education level, and many of them have tertiary education level.
    - Most of them does not have credit in default, housing loan or personal loan.
    - They are mostly contacted by cellular.
    - Most clients do not have previous campaign before.
    

## Model

In data preprocessing, we standardize the numerical features and use one-hot-encoding to convert categorical features into dummy variables. We used some traditional classification models for dealing with this classification problem such as logistic regression, support vector machine and random forest from `sklearn` package. To deal with the data imbalance, we tuned the hyper-parameter `class_weight` in built-in sklearn model parameter. Apart from that, we used one of the variations of Gradient Boosting Trees: LightGBM, as more complex solution to the problem.

We used F1 score as our main evaluation metric. Given the formula $F1_{score} = 2 * \frac{(precision * recall)}{(precision + recall)}$, it will try to find a balance between the precision and recall, which is very useful in our imbalance case.

For finding the best model, we tried `GridSearchCV` approach from `sklearn` package.

# Results & Discussion

## Results

We trained different variations of the above mentioned `sklearn` models. The performance of all models tested with `GridSearchCV` can be seen in the below chart:

<img src="../reports/training_report.png" width="400" height="400">
Figure 3: Training report

Please note, however, that the LightGBM is not tuned using `GridSearchCV` approach. Moreover, the results shown are from `GridSearchCV` output itself, so `test_scores` is actually lower than what is displayed. 

Apart from that, we also show results in `csv` file. Here, all the results are from true `train` and `test` datasets and can be compared fairly.

In [3]:
import pandas as pd
pd.read_csv('../reports/training_report.csv').drop('Unnamed: 0', axis=1)

,Model name,Best parameters,Train F1,Test F1,Test accuracies
0,LogisticRegression,"{'C': 0.1, 'penalty': 'l1', 'solver': 'libline...",0.521463,"(0.49333333333333323,)","(0.8320441988950277,)"
1,SVC,"{'C': 0.1, 'degree': 3, 'kernel': 'poly'}",0.562607,"(0.5333333333333333,)","(0.8607734806629834,)"
2,RandomForestClassifier,"{'criterion': 'entropy', 'max_depth': 16, 'n_e...",0.972781,"(0.3757575757575758,)","(0.8861878453038674,)"
3,Booster,"{'learning_rate': 0.1, 'lambda_l1': 0.5, 'max_...",0.622468,0.4129032258064516,0.8994475138121547


As we can see, the best Test `F1` score gave SVM, even though the best accuracy has `LightGBM`. This is the result of class imbalance problem. As `F`1 metric handles the best that problem, we chose `SVM` as the best model.

## Further directions

For future implementation, we can do further feature selection and feature engineering for better model performance. Also, we can implement some other ways to deal with the imbalance dataset, such as oversampling (SMOTE) and down-sampling methods.

# Reference


[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

UCI Machine Learning Repository. University of California, Irvine, School of Information; Computer Sciences.https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Raschka, S. and Mirjalili, V., 2019. Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow 2. Packt Publishing Ltd.

Bisong, Ekaba. 2019. “Matplotlib and Seaborn.” In Building Machine Learning and Deep Learning Models on Google Cloud Platform, 151–65. Springer.

Chandra, Rakesh Vidya, and Bala Subrahmanyam Varanasi. 2015. Python Requests Essentials. Packt Publishing Ltd.

Costa-Luis, Casper da. 2019. “Tqdm: A Fast, Extensible Progress Meter for Python and Cli.” Journal of Open Source Software 4 (37): 1277.

de Jonge, Edwin. 2018. Docopt: Command-Line Interface Specification Language. https://CRAN.R-project.org/package=docopt.

Ke, Guolin, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. “Lightgbm: A Highly Efficient Gradient Boosting Decision Tree.” In Advances in Neural Information Processing Systems, 3146–54.

Keleshev, Vladimir. 2014. Docopt: Command-Line Interface Description Language. https://github.com/docopt/docopt.

McKinney, Wes, and others. 2011. “Pandas: A Foundational Python Library for Data Analysis and Statistics.” Python for High Performance and Scientific Computing 14 (9).

Oliphant, Travis E. 2006. A Guide to Numpy. Vol. 1. Trelgol Publishing USA.

Pedregosa, Fabian, Gaël Varoquaux, Alexandre Gramfort, Vincent Michel, Bertrand Thirion, Olivier Grisel, Mathieu Blondel, et al. 2011. “Scikit-Learn: Machine Learning in Python.” Journal of Machine Learning Research 12 (Oct): 2825–30.

R Core Team. 2019. R: A Language and Environment for Statistical Computing. Vienna, Austria: R Foundation for Statistical Computing. https://www.R-project.org/.

Reitz,Kenneth.2019._Requests: HTTP for Humans_.https://pypi.org/project/requests/.

Salunke, Sagar Shivaji. 2014. Selenium Webdriver in Python: Learn with Examples. CreateSpace Independent Publishing Platform.

Van Rossum, Guido, and Fred L. Drake. 2009. Python 3 Reference Manual. Scotts Valley, CA: CreateSpace.

VanderPlas, Jacob, Brian Granger, Jeffrey Heer, Dominik Moritz, Kanit Wongsuphasawat, Arvind Satyanarayan, Eitan Lees, Ilia Timofeev, Ben Welsh, and Scott Sievert. 2018. “Altair: Interactive Statistical Visualizations for Python.” Journal of Open Source Software 3 (32): 1057.

Wickham, Hadley. 2011. “Testthat: Get Started with Testing.” The R Journal 3: 5–10. https://journal.r-project.org/archive/2011-1/RJournal_2011-1_Wickham.pdf.